In [1]:
import numpy as np
import librosa
import random

def manipulate0(data, noise_factor):
    noise = np.random.randn(len(data))
    augmented_data = data + noise_factor * noise
    # Cast back to same data type
    augmented_data = augmented_data.astype(type(data[0]))
    return augmented_data

def manipulate1(data, sampling_rate, shift_max, shift_direction):
    shift = np.random.randint(sampling_rate * shift_max)
    if shift_direction == 'right':
        shift = -shift
    elif shift_direction == 'both':
        direction = np.random.randint(0, 2)
        if direction == 1:
            shift = -shift
    augmented_data = np.roll(data, shift)
    # Set to silence for heading/ tailing
    if shift > 0:
        augmented_data[:shift] = 0
    else:
        augmented_data[shift:] = 0
    return augmented_data

def manipulate2(data, sampling_rate, pitch_factor):
    return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)

def manipulate3(data, speed_factor):
    return librosa.effects.time_stretch(data, speed_factor)

In [2]:
import os

import librosa
from librosa.core.convert import mel_frequencies
import pandas as pd


import numpy as np
from tqdm import tqdm

import torch
from torch import flatten
from torch.nn import Conv2d, Linear, LogSoftmax, MaxPool2d, Module, ReLU
from torch.utils.data import Dataset


audio_dataset_path = "C:/Users/Yashvi/Desktop/TestCases/"
metadata = pd.read_csv("C:/Users/Yashvi/Desktop/speech_preprocessing.csv")
metadata.head()

kwargs = {"htk": True, "norm": "slaney"}

def switch_function(signal,sr):
    n = random.randint(0,3)
    M0 = [.0050,.0015,.0025,.0060,.0040]
    M1 = [0.1,0.2,0.15,0.25,0.3]
    M2 = [0.5,1,1.5,2,2.5]
    M3 = [0.5,0.75,1,1.15,1.25]
    signal={
        0: manipulate0(signal,random.choice(M0)),
        1: manipulate1(signal,sr,random.choice(M1),'both'),
        2: manipulate2(signal,sr,random.choice(M2)),
        3: manipulate3(signal,random.choice(M3))
    }
    return signal.get(n,"Invalid")


def features_extractor(signal,sr):
    preemphasized_signal = librosa.effects.preemphasis(signal, coef=0.95)
    mel_spectrogram = librosa.feature.melspectrogram(
        y=preemphasized_signal,
        sr=44100,
        n_mels=42,
        n_fft=512,
        hop_length=100,
        win_length=256,
        window="hann",
        center=True,
        pad_mode="reflect",
        power=2.0,
        **kwargs
    )

    mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel_spectrogram), n_mfcc=12, dct_type=2)
    mfcc_scaled_features = np.mean(mfcc.T, axis=0)

    return [mfcc_scaled_features, mfcc]



extracted_features = []
spectogram_data = []
for index_num, row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), str(row["file_name"]))
    gender_labels = row["gender"]
    age_group_labels = row["age_group"]
    state_labels = row["state"]
    heart_rate_labels = row["heart_rate"]
    signal, sr = librosa.load(file_name, sr=44100)
    data1 = features_extractor(signal,sr)
    signal1 = switch_function(signal,sr)
    data2 = features_extractor(signal1,sr)
    signal2 = switch_function(signal,sr)
    data3 = features_extractor(signal2,sr)
    spectogram_data.append(data1[1])
    extracted_features.append(
        [data1[0], gender_labels, age_group_labels, state_labels, heart_rate_labels]
    )
    extracted_features.append(
        [data2[0], gender_labels, age_group_labels, state_labels, heart_rate_labels]
    )
    extracted_features.append(
        [data3[0], gender_labels, age_group_labels, state_labels, heart_rate_labels]
    )

one_hot = pd.get_dummies(metadata["gender"])
# Drop column B as it is now encoded
metadata = metadata.drop("gender", axis=1)
# Join the encoded df
metadata = metadata.join(one_hot)

newstates= []
states = metadata["state"].to_list()
for state in states:
    newstates.append(
        state.lower().strip()
    )
metadata = metadata.drop("state", axis=1)
metadata['state'] = newstates

one_hot = pd.get_dummies(metadata["state"])
metadata = metadata.drop("state", axis=1)
metadata = metadata.join(one_hot)

one_hot = pd.get_dummies(metadata["age_group"])
metadata = metadata.drop("age_group", axis=1)
metadata = metadata.join(one_hot)

heart_rates = metadata["heart_rate"].to_list()

metadata = metadata.drop("heart_rate", axis=1)
metadata = metadata.drop("file_name", axis=1)


remaining_features = metadata.values

result = []

for i in range(len(extracted_features)):
    features = extracted_features[i][0].tolist()
    new = []

    new += extracted_features[i][0].tolist()
    new += remaining_features[i%97].tolist()
    new.append(heart_rates[i%97])

    result.append(new)

names = []
for i in range(1,13):
    names.append(f"mfcc_{i}")
s = "F 	M 	after_workout 	happy 	neutral 	relaxed 	stressed 	tired 	20-29 	30-39 	40-49 	50-59 	60-69"

names = names + s.strip().split()
names.append("heart_rate")

pd.DataFrame(result, columns=names).to_csv("save.csv", index=False)

97it [04:16,  2.65s/it]


In [54]:
print(extracted_features[0])
print(remaining_features[0])

[array([-424.3054   ,   28.069408 ,   -0.9381784,   34.194496 ,
        -15.37882  ,   13.257497 ,  -15.704871 ,    3.3506052,
         -6.0990524,    2.1145062,    3.0280347,   -6.034267 ],
      dtype=float32), 'F', '20-29', 'tired', 80]
[1 0 0 0 0 0 0 1 1 0 0 0 0]


In [55]:
metadata.head() 

,F,M,after_workout,happy,neutral,relaxed,stressed,tired,20-29,30-39,40-49,50-59,60-69
0,1,0,0,0,0,0,0,1,1,0,0,0,0
1,0,1,0,0,0,0,0,1,1,0,0,0,0
2,0,1,1,0,0,0,0,0,1,0,0,0,0
3,1,0,0,0,0,0,0,1,1,0,0,0,0
4,0,1,0,1,0,0,0,0,1,0,0,0,0


In [62]:
import torch
from torch import flatten
from torch.nn import Conv2d, Linear, LogSoftmax, MaxPool2d, Module, ReLU
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch.optim import Adam
import time

class yashvidataset(Dataset):
    def __init__(self, path, train=True, split = 0.8):
        df = pd.read_csv(path)
        if train:
            self.length = int(len(df) * split)
            values = df.values[:self.length]
        else:
            self.training_length = int(len(df) * split)
            values = df.values[self.training_length:]
            self.length = len(df) - self.training_length

        self.build_data(values)

    def build_data(self, values):
        self.X = []
        self.Y = []

        for value in values:
            self.X.append(value[:len(value)-1])
            self.Y.append(value[-1])
        
    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        feature = torch.FloatTensor(self.X[idx])
        label = self.Y[idx]

        return feature, label


training_dataset = yashvidataset("save.csv", train=True)
testing_dataset = yashvidataset("save.csv", train=False)

BATCH_SIZE = 10

train_dataloader = DataLoader(training_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(testing_dataset, batch_size=BATCH_SIZE, shuffle=False)

trainSteps = len(train_dataloader.dataset) // BATCH_SIZE
testSteps = len(test_dataloader.dataset) // BATCH_SIZE

import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(25, 50) 
        self.drop1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(50, 100)
        self.drop2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(100, 1)

    def forward(self, x):
        x = self.drop1(F.relu(self.fc1(x)))
        x = self.drop2(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

INIT_LR = 1e-5
EPOCHS = 1000


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Net().to(device)


opt = Adam(model.parameters(), lr=INIT_LR)
lossFn = nn.MSELoss()
H = {
"train_loss": [],
"test_loss": [],
}

print("[INFO] Training the network...")
startTime = time.time()


for e in range(0, EPOCHS):
    #training mode
    model.train()

    totalTrainLoss = 0
    totalTestLoss = 0
   
    trainCorrect = 0
    testCorrect = 0
    
    for (x, y) in train_dataloader:
        (x, y) = (x.to(device), y.to(device))
        
        pred = model(x)
        loss = lossFn(pred.float(), y.float())
        # zero out the gradients, perform the backpropagation step,
        # and update the weights
        opt.zero_grad()
        loss.backward()
        opt.step()
        # loss to the total training loss 
        totalTrainLoss += loss

    # switch off autograd for evaluation
    with torch.no_grad():
        # set the model in evaluation mode
        model.eval()
        # loop over the validation set
        for (x, y) in test_dataloader:
            # send the input to the device
            (x, y) = (x.to(device), y.to(device))
            # make the predictions and calculate the validation loss
            pred = model(x)
            totalTestLoss += lossFn(pred, y)
            # calculate the number of correct predictions

    # calculate the average training and validation loss
    avgTrainLoss = totalTrainLoss / trainSteps
    avgTestLoss = totalTestLoss / testSteps

    # update our training history
    H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
    H["test_loss"].append(avgTestLoss.cpu().detach().numpy())
    # print the model training and validation information
    print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
    print("Train loss: {:.6f}".format(avgTrainLoss))
    print("Test loss: {:.6f}\n".format(avgTestLoss))

[INFO] training the network...
[INFO] EPOCH: 1/1000
Train loss: 7770.501953
Test loss: 7753.899273

[INFO] EPOCH: 2/1000
Train loss: 7634.820801
Test loss: 7604.985752

[INFO] EPOCH: 3/1000
Train loss: 7573.990723
Test loss: 7455.704735

[INFO] EPOCH: 4/1000
Train loss: 7398.588867
Test loss: 7312.229932

[INFO] EPOCH: 5/1000
Train loss: 7144.687988
Test loss: 7168.286338

[INFO] EPOCH: 6/1000
Train loss: 7230.029785
Test loss: 7028.167567

[INFO] EPOCH: 7/1000
Train loss: 7031.935547
Test loss: 6889.491442

[INFO] EPOCH: 8/1000
Train loss: 6997.109863
Test loss: 6750.193522

[INFO] EPOCH: 9/1000
Train loss: 6797.010742
Test loss: 6614.286667

[INFO] EPOCH: 10/1000
Train loss: 6751.740723
Test loss: 6478.309097

[INFO] EPOCH: 11/1000
Train loss: 6599.065430
Test loss: 6343.974789

[INFO] EPOCH: 12/1000
Train loss: 6574.500000
Test loss: 6211.440276

[INFO] EPOCH: 13/1000
Train loss: 6352.411133
Test loss: 6080.447947

[INFO] EPOCH: 14/1000
Train loss: 6128.214844
Test loss: 5946.958857

[INFO] EPOCH: 125/1000
Train loss: 373.403564
Test loss: 275.050999

[INFO] EPOCH: 126/1000
Train loss: 429.442719
Test loss: 275.721695

[INFO] EPOCH: 127/1000
Train loss: 441.657745
Test loss: 276.043238

[INFO] EPOCH: 128/1000
Train loss: 442.676544
Test loss: 276.460522

[INFO] EPOCH: 129/1000
Train loss: 441.080292
Test loss: 278.391325

[INFO] EPOCH: 130/1000
Train loss: 402.909973
Test loss: 277.073005

[INFO] EPOCH: 131/1000
Train loss: 402.236877
Test loss: 276.225263

[INFO] EPOCH: 132/1000
Train loss: 428.996979
Test loss: 277.748851

[INFO] EPOCH: 133/1000
Train loss: 433.479309
Test loss: 280.343580

[INFO] EPOCH: 134/1000
Train loss: 434.452332
Test loss: 281.579165

[INFO] EPOCH: 135/1000
Train loss: 397.002197
Test loss: 282.183731

[INFO] EPOCH: 136/1000
Train loss: 447.361237
Test loss: 280.477152

[INFO] EPOCH: 137/1000
Train loss: 444.902740
Test loss: 281.304896

[INFO] EPOCH: 138/1000
Train loss: 438.492004
Test loss: 281.715890

[INFO] EPOCH: 139/1000
Train loss:

Test loss: 220.354815

[INFO] EPOCH: 246/1000
Train loss: 355.331970
Test loss: 219.971843

[INFO] EPOCH: 247/1000
Train loss: 347.721924
Test loss: 218.810474

[INFO] EPOCH: 248/1000
Train loss: 360.367767
Test loss: 218.674485

[INFO] EPOCH: 249/1000
Train loss: 366.737640
Test loss: 218.949077

[INFO] EPOCH: 250/1000
Train loss: 364.861969
Test loss: 218.248176

[INFO] EPOCH: 251/1000
Train loss: 335.738953
Test loss: 219.091670

[INFO] EPOCH: 252/1000
Train loss: 358.591217
Test loss: 217.998563

[INFO] EPOCH: 253/1000
Train loss: 367.871979
Test loss: 219.006007

[INFO] EPOCH: 254/1000
Train loss: 366.463013
Test loss: 219.011643

[INFO] EPOCH: 255/1000
Train loss: 313.323639
Test loss: 221.445654

[INFO] EPOCH: 256/1000
Train loss: 387.680206
Test loss: 220.078283

[INFO] EPOCH: 257/1000
Train loss: 380.633270
Test loss: 222.637702

[INFO] EPOCH: 258/1000
Train loss: 358.745636
Test loss: 223.422413

[INFO] EPOCH: 259/1000
Train loss: 355.039856
Test loss: 222.481664

[INFO] EPOC

Test loss: 202.039635

[INFO] EPOCH: 368/1000
Train loss: 323.773712
Test loss: 202.064359

[INFO] EPOCH: 369/1000
Train loss: 361.721222
Test loss: 202.271211

[INFO] EPOCH: 370/1000
Train loss: 319.548157
Test loss: 202.361018

[INFO] EPOCH: 371/1000
Train loss: 354.052155
Test loss: 202.123400

[INFO] EPOCH: 372/1000
Train loss: 313.843933
Test loss: 202.221757

[INFO] EPOCH: 373/1000
Train loss: 323.150635
Test loss: 202.174442

[INFO] EPOCH: 374/1000
Train loss: 320.456726
Test loss: 201.958064

[INFO] EPOCH: 375/1000
Train loss: 329.505188
Test loss: 202.082085

[INFO] EPOCH: 376/1000
Train loss: 337.825012
Test loss: 202.361409

[INFO] EPOCH: 377/1000
Train loss: 339.201569
Test loss: 202.678265

[INFO] EPOCH: 378/1000
Train loss: 340.102203
Test loss: 203.296158

[INFO] EPOCH: 379/1000
Train loss: 382.040070
Test loss: 203.064353

[INFO] EPOCH: 380/1000
Train loss: 291.692078
Test loss: 203.475296

[INFO] EPOCH: 381/1000
Train loss: 306.453430
Test loss: 203.087478

[INFO] EPOC

[INFO] EPOCH: 489/1000
Train loss: 309.164948
Test loss: 199.549528

[INFO] EPOCH: 490/1000
Train loss: 312.819305
Test loss: 199.586901

[INFO] EPOCH: 491/1000
Train loss: 316.622528
Test loss: 199.681186

[INFO] EPOCH: 492/1000
Train loss: 322.421661
Test loss: 199.758574

[INFO] EPOCH: 493/1000
Train loss: 310.615204
Test loss: 199.792803

[INFO] EPOCH: 494/1000
Train loss: 320.634583
Test loss: 199.666175

[INFO] EPOCH: 495/1000
Train loss: 307.230835
Test loss: 199.730616

[INFO] EPOCH: 496/1000
Train loss: 296.568878
Test loss: 200.041684

[INFO] EPOCH: 497/1000
Train loss: 308.231354
Test loss: 199.721932

[INFO] EPOCH: 498/1000
Train loss: 323.415131
Test loss: 199.771036

[INFO] EPOCH: 499/1000
Train loss: 304.405182
Test loss: 199.658870

[INFO] EPOCH: 500/1000
Train loss: 301.351685
Test loss: 199.602326

[INFO] EPOCH: 501/1000
Train loss: 297.517578
Test loss: 199.679609

[INFO] EPOCH: 502/1000
Train loss: 339.016357
Test loss: 199.654545

[INFO] EPOCH: 503/1000
Train loss:

[INFO] EPOCH: 611/1000
Train loss: 293.561920
Test loss: 201.494269

[INFO] EPOCH: 612/1000
Train loss: 298.989380
Test loss: 202.065144

[INFO] EPOCH: 613/1000
Train loss: 279.163788
Test loss: 202.179294

[INFO] EPOCH: 614/1000
Train loss: 280.191589
Test loss: 202.351974

[INFO] EPOCH: 615/1000
Train loss: 289.477386
Test loss: 201.536040

[INFO] EPOCH: 616/1000
Train loss: 301.764496
Test loss: 201.669372

[INFO] EPOCH: 617/1000
Train loss: 285.709229
Test loss: 200.613788

[INFO] EPOCH: 618/1000
Train loss: 298.977325
Test loss: 201.050660

[INFO] EPOCH: 619/1000
Train loss: 304.052673
Test loss: 201.228637

[INFO] EPOCH: 620/1000
Train loss: 286.990631
Test loss: 201.710309

[INFO] EPOCH: 621/1000
Train loss: 293.604340
Test loss: 202.471039

[INFO] EPOCH: 622/1000
Train loss: 304.543915
Test loss: 202.396754

[INFO] EPOCH: 623/1000
Train loss: 279.232910
Test loss: 201.534850

[INFO] EPOCH: 624/1000
Train loss: 297.023438
Test loss: 202.052910

[INFO] EPOCH: 625/1000
Train loss:

[INFO] EPOCH: 734/1000
Train loss: 283.810669
Test loss: 204.590914

[INFO] EPOCH: 735/1000
Train loss: 292.556091
Test loss: 204.211703

[INFO] EPOCH: 736/1000
Train loss: 276.505127
Test loss: 205.002826

[INFO] EPOCH: 737/1000
Train loss: 271.153229
Test loss: 204.665037

[INFO] EPOCH: 738/1000
Train loss: 274.749695
Test loss: 203.396605

[INFO] EPOCH: 739/1000
Train loss: 281.880615
Test loss: 205.176490

[INFO] EPOCH: 740/1000
Train loss: 272.952332
Test loss: 205.077352

[INFO] EPOCH: 741/1000
Train loss: 285.557312
Test loss: 204.561773

[INFO] EPOCH: 742/1000
Train loss: 342.020599
Test loss: 204.424233

[INFO] EPOCH: 743/1000
Train loss: 312.593689
Test loss: 204.182192

[INFO] EPOCH: 744/1000
Train loss: 289.383972
Test loss: 205.485657

[INFO] EPOCH: 745/1000
Train loss: 286.158295
Test loss: 205.758214

[INFO] EPOCH: 746/1000
Train loss: 308.774841
Test loss: 205.823537

[INFO] EPOCH: 747/1000
Train loss: 286.534210
Test loss: 205.891073

[INFO] EPOCH: 748/1000
Train loss:

[INFO] EPOCH: 856/1000
Train loss: 287.084290
Test loss: 208.396684

[INFO] EPOCH: 857/1000
Train loss: 281.219696
Test loss: 208.503885

[INFO] EPOCH: 858/1000
Train loss: 292.760437
Test loss: 208.849567

[INFO] EPOCH: 859/1000
Train loss: 256.215820
Test loss: 206.395945

[INFO] EPOCH: 860/1000
Train loss: 295.991882
Test loss: 204.889434

[INFO] EPOCH: 861/1000
Train loss: 266.560028
Test loss: 205.708501

[INFO] EPOCH: 862/1000
Train loss: 263.594879
Test loss: 205.596798

[INFO] EPOCH: 863/1000
Train loss: 283.477234
Test loss: 205.539378

[INFO] EPOCH: 864/1000
Train loss: 281.859253
Test loss: 205.570158

[INFO] EPOCH: 865/1000
Train loss: 300.909546
Test loss: 207.558950

[INFO] EPOCH: 866/1000
Train loss: 291.191956
Test loss: 210.086631

[INFO] EPOCH: 867/1000
Train loss: 281.949738
Test loss: 209.792857

[INFO] EPOCH: 868/1000
Train loss: 278.898376
Test loss: 208.743397

[INFO] EPOCH: 869/1000
Train loss: 298.313690
Test loss: 211.944344

[INFO] EPOCH: 870/1000
Train loss:

[INFO] EPOCH: 978/1000
Train loss: 263.398193
Test loss: 213.682314

[INFO] EPOCH: 979/1000
Train loss: 300.857544
Test loss: 214.750511

[INFO] EPOCH: 980/1000
Train loss: 261.352173
Test loss: 215.293347

[INFO] EPOCH: 981/1000
Train loss: 275.738739
Test loss: 214.461477

[INFO] EPOCH: 982/1000
Train loss: 284.808624
Test loss: 214.291360

[INFO] EPOCH: 983/1000
Train loss: 292.233063
Test loss: 215.743840

[INFO] EPOCH: 984/1000
Train loss: 266.767761
Test loss: 213.706972

[INFO] EPOCH: 985/1000
Train loss: 264.936615
Test loss: 214.867499

[INFO] EPOCH: 986/1000
Train loss: 285.621399
Test loss: 212.967822

[INFO] EPOCH: 987/1000
Train loss: 282.645599
Test loss: 214.068923

[INFO] EPOCH: 988/1000
Train loss: 282.901398
Test loss: 214.501793

[INFO] EPOCH: 989/1000
Train loss: 270.321106
Test loss: 215.260271

[INFO] EPOCH: 990/1000
Train loss: 243.239532
Test loss: 214.691002

[INFO] EPOCH: 991/1000
Train loss: 275.591064
Test loss: 215.926865

[INFO] EPOCH: 992/1000
Train loss:

In [50]:
print(pred,y)

tensor([[84.8456],
        [93.5064],
        [78.3952]]) tensor([97., 81., 97.], dtype=torch.float64)


In [63]:
PATH = 'C:/Users/Yashvi/Desktop/Model1.pth'
torch.save(model.state_dict(), PATH)

In [65]:
model = Net()
model.load_state_dict(torch.load(PATH))
model.eval()

Net(
  (fc1): Linear(in_features=25, out_features=50, bias=True)
  (drop1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=50, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=1, bias=True)
)